In [2]:
import opencorpora

file_path = "dataset_37845_1.txt"

In [3]:
odict_to_pos = {
    'вводн.': 'ADV',
    'ж': 'S',
    'жо': 'S',
    'м': 'S',
    'межд.': 'ADV',
    'мн.': 'S',
    'мо': 'S',
    'мо-жо': 'S',
    'мс-п': 'ADV',
    'н': 'ADV',
    'нсв': 'V',
    'п': 'A',
    'предик.': 'ADV',
    'предл.': 'PR',
    'с': 'S',
    'св': 'V',
    'св-нсв': 'V',
    'со': 'S',
    'союз': 'CONJ',
    'сравн.': 'ADV',
    'част.': 'ADV',
    'числ.': 'ADV',
    'числ.-п': 'A'
}

In [43]:
odict_words_lemmas = {}
odict_words_pos = {}

with open('odict.csv', encoding='1251') as odict:
    for line in odict:
        items = line.split(',')[:-1]
        lemma, pos = items[:2]
        words = items[2:]
        words.append(lemma)
        pos = odict_to_pos[pos]

        for word in words:
            word = word.strip().lower()
            if word not in odict_words_lemmas:
                odict_words_lemmas[word] = {}
            odict_words_lemmas[word][lemma] = odict_words_lemmas[word].get(lemma, 0) + 1
            if word not in odict_words_pos:
                odict_words_pos[word] = {}
            odict_words_pos[word][pos] = odict_words_pos[word].get(pos, 0) + 1

In [5]:
corpus = opencorpora.load('annot.opcorpora.xml')

In [6]:
ocorpora_to_pos = {
    'NOUN': 'S',
    'ADJF': 'A',
    'ADJS': 'A',
    'VERB': 'V',
    'INFN': 'V',
    'PRTF': 'V',
    'PRTS': 'V',
    'GRND': 'V',
    'ADVB': 'ADV',
    'NPRO': 'S',
    'PREP': 'PR',
    'CONJ': 'CONJ',
    'PRCL': 'ADV',
    'INTJ': 'ADV'
}

In [49]:
ocorp_words_lemmas = {}
ocorp_words_pos = {}

n = 0
for doc in corpus:
    for sentence in doc.sentences:
        for token in sentence.tokens:
            n += 1
            word = token.source.strip().lower()
            lemma = token.parse.lemma.strip().lower()
            pos = token.parse.grammemes[0]
            pos = ocorpora_to_pos.get(pos, 'UNK')

            if word not in ocorp_words_lemmas:
                ocorp_words_lemmas[word] = {}
            ocorp_words_lemmas[word][lemma] = ocorp_words_lemmas[word].get(lemma, 0) + 1
            if word not in ocorp_words_pos:
                ocorp_words_pos[word] = {}
            ocorp_words_pos[word][pos] = ocorp_words_pos[word].get(pos, 0) + 1 

In [28]:
def arg_max_dict(d, ignore=set('UNK')):
    max_v = 0
    max_a = None
    s = 0
    for k in d:
        if k in ignore:
            continue
        s += d[k]
        if max_v < d[k]:
            max_v = d[k]
            max_a = k
    return max_a #, max_v / s

In [24]:
def two_dict_and_default(key, d1, d2, default):
    if key in d1:
        answer = arg_max_dict(d1[key])
    elif key in d2:
        answer = arg_max_dict(d2[key])
    else:
        answer = default
    return answer

In [57]:
sents = []
with open(file_path) as f:
    for line in f:
        sents.append("".join(c for c in line if c not in '.,?!').strip().split(' '))

In [58]:
output = []
for words in sents:
    sent = []
    for word in words:
        w = word.lower()
#         lemma = two_dict_and_default(w, odict_words_lemmas, ocorp_words_lemmas, word)
#         pos = two_dict_and_default(w, odict_words_pos, ocorp_words_pos, 'S')
        
#         lemma = two_dict_and_default(w, ocorp_words_lemmas, odict_words_lemmas, w)
#         pos = two_dict_and_default(w, ocorp_words_pos, odict_words_pos, 'S')

        lemma = two_dict_and_default(w, odict_words_lemmas, ocorp_words_lemmas, w)
        pos = two_dict_and_default(w, ocorp_words_pos, odict_words_pos, 'S')
        
        sent.append(word + '{' + lemma + '=' + pos + '}')
    output.append(" ".join(sent))

In [59]:
with open("out.txt", 'w') as f:
    for s in output:
        f.write(s + "\n")

In [53]:
odict_words_lemmas["в"]

{'в': 1}